In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('hw1_result_df.csv')
df.head()

,document,question,answer,source,document_type,document_length,question_length,answer_length,answer_to_document_ratio,question_ru,answer_ru,document_ru,question_type
0,the world's first institution of technology or...,what year was the banská akadémia founded?,1735,SQuAD,Wikipedia Article,608,42,4,0.006579,В каком году была основана банска академия?,1735,Первый в мире технологический или технический ...,как
1,the standard specifies how speed ratings shoul...,what is another speed that can also be reporte...,sos-based speed,SQuAD,Wikipedia Article,890,62,15,0.016854,"какова еще одна скорость, о которой может быть...",Скорость на основе sos,"В стандарте указывается, каким образом показан...",как
2,the most impressive and famous of sumerian bui...,where were the use of advanced materials and t...,sumerian temples and palaces,SQuAD,Wikipedia Article,578,76,28,0.048443,"где используются передовые материалы и методы,...",Сумеричные храмы и дворцы,Наиболее впечатляющими и известными из сумероч...,где
3,ann arbor has a council-manager form of govern...,who is elected every even numbered year?,mayor,SQuAD,Wikipedia Article,841,40,5,0.005945,Кто избирается каждый четный год?,Мэр,Муниципальный совет состоит из 11 членов: мэра...,кто
4,"shortly before his death, when he was already ...",what was the purpose of top secret icbm commit...,decide on the feasibility of building an icbm ...,SQuAD,Wikipedia Article,658,50,90,0.136778,Какова была цель сверхсекретного комитета по ц...,:: принять решение о возможности строительства...,"Незадолго до своей смерти фон Неуманн, когда о...",как


**Здесь данные уже условно размечены вопрос-ответом, поэтому давайте дособерем в датасет дополнительных неразмеченных сэмплов:**

In [3]:
from hw3_scripts import *

In [4]:
def create_test_cases(n=10):
    """Создаем тестовые кейсы с известными правильными ответами"""
    test_cases = []
    
    test_documents = [
        "Искусственный интеллект (ИИ) — это способность компьютерных систем выполнять задачи, которые обычно требуют человеческого интеллекта. Сюда входят обучение, рассуждение, решение проблем, понимание языка и восприятие.",
        "Глубокое обучение — подраздел машинного обучения, основанный на искусственных нейронных сетях с многими слоями. Оно позволяет моделям обучаться представлению данных с несколькими уровнями абстракции.",
        "Обработка естественного языка (NLP) — область искусственного интеллекта, которая фокусируется на взаимодействии между компьютерами и человеческим языком. Основные задачи включают понимание текста, генерацию текста и перевод."
    ]
    
    for i, doc in enumerate(test_documents):
        question1 = f"Что такое {doc.split(' ')[0]} {doc.split(' ')[1]}?"
        answer1 = doc.split('.')[0] + "."
        
        question2 = f"Какие основные задачи включает {doc.split(' ')[0]} {doc.split(' ')[1]}?"

        for sentence in doc.split('.'):
            if any(keyword in sentence.lower() for keyword in ["включает", "входят", "основные задачи"]):
                answer2 = sentence.strip() + "."
                break
        else:
            answer2 = "Информация не представлена в тексте."
        
        test_cases.append({
            'document': doc,
            'question': question1,
            'correct_answer': answer1,
            'document_ru': doc,
            'question_ru': question1,
            'answer_ru': '',  ### будет заполнено разметчиком
            'is_test_case': True,
            'document_type': 'test_case',
            'source': 'validation',
            'document_length': len(doc),
            'question_length': len(question1),
            'question_type': 'factual'
        })
        
        test_cases.append({
            'document': doc,
            'question': question2,
            'correct_answer': answer2,
            'document_ru': doc,
            'question_ru': question2,
            'answer_ru': '',  ### будет заполнено разметчиком
            'is_test_case': True,
            'document_type': 'test_case',
            'source': 'validation',
            'document_length': len(doc),
            'question_length': len(question2),
            'question_type': 'descriptive'
        })
    
    return test_cases[:n]  # возвращаем только n кейсов

In [5]:
print("Загружаем новые данные...")

arxiv_data = get_new_arxiv_abstracts(num_articles=30, search_query='artificial intelligence')
wiki_data = get_new_wikipedia_articles(num_articles=30)
    
print(f"Получено {len(arxiv_data)} статей с arXiv")
print(f"Получено {len(wiki_data)} статей из Википедии")
    
test_cases = create_test_cases(n=10)
print(f"Создано {len(test_cases)} тестовых кейсов")
    
new_data = pd.DataFrame(arxiv_data + wiki_data + test_cases)
    
new_data['question'] = None
new_data['answer'] = None
if 'question_ru' not in new_data.columns:
        new_data['question_ru'] = None
if 'answer_ru' not in new_data.columns:
        new_data['answer_ru'] = None

for col in df.columns:
        if col not in new_data.columns:
            if col in ['question_length', 'answer_length']:
                new_data[col] = 0
            elif col == 'answer_to_document_ratio':
                new_data[col] = 0.0
            elif col == 'question_type' and 'question_type' not in new_data.columns:
                new_data[col] = None

Загружаем новые данные...


Загрузка статей с arXiv:   0%|          | 0/30 [00:00<?, ?it/s]

Загрузка статей из Википедии:   0%|          | 0/30 [00:00<?, ?it/s]

Получено 30 статей с arXiv
Получено 29 статей из Википедии
Создано 6 тестовых кейсов


In [6]:
# Создаем датасет для разметки
unlabeled_data = new_data.copy()
    
print(f"Размер неразмеченного датасета: {len(unlabeled_data)}")
    
# Сохраняем неразмеченный датасет
unlabeled_data.to_csv('unlabeled_dataset.csv', index=False)

Размер неразмеченного датасета: 65


## ТЗ

Хороший инстракт к разметчикам должен быть исчерпывающим и не допускать двойных толкований.
Нужно давать примеры и пояснять edge-кейсы.

In [7]:
tz_text = """
# Техническое задание для разметки QA-датасета для научных текстов

## Цель задания
Разметить набор научных текстов и статей из Википедии путем создания вопросов к тексту и соответствующих ответов, которые можно найти непосредственно в тексте.

## Что нужно сделать
1. Для каждого документа в датасете необходимо:
   - Прочитать и понять содержание текста
   - Создать 1-2 содержательных вопроса на русском языке к тексту
   - Предоставить ответ на каждый вопрос, используя фрагмент исходного текста

## Требования к вопросам
1. Вопросы должны быть сформулированы на русском языке
2. Каждый вопрос должен быть четким и конкретным
3. Вопрос должен относиться к фактам, концепциям или информации, содержащейся в тексте
4. Вопросы должны быть разных типов:
   - Фактические: запрашивают конкретную информацию (даты, имена, определения)
   - Описательные: требуют более развернутого ответа о процессах, методах, причинах
   - Сравнительные: просят сравнить несколько концепций или объектов

## Требования к ответам
1. Ответ должен содержаться в тексте документа (копи-паста из текста или минимальное перефразирование)
2. Ответ должен быть полным и достаточным для заданного вопроса
3. Длина ответа: от 1 предложения до 3-х предложений (не более 500 символов)
4. Если точного ответа нет в тексте, указать "Информация не представлена в тексте"

## Примеры правильной разметки

### Пример 1 (научный текст):
**Документ:** 
"Трансформеры — это архитектура нейронных сетей, основанная на механизме внимания. Они были представлены в статье "Attention is All You Need" от Google в 2017 году. Трансформеры обладают высокой эффективностью для задач обработки естественного языка и машинного перевода благодаря своей способности параллельно обрабатывать входные последовательности."

**Вопрос (фактический):** Когда и кем были представлены трансформеры?
**Ответ:** Они были представлены в статье "Attention is All You Need" от Google в 2017 году.

**Вопрос (описательный):** Почему трансформеры эффективны для задач обработки естественного языка?
**Ответ:** Трансформеры обладают высокой эффективностью для задач обработки естественного языка и машинного перевода благодаря своей способности параллельно обрабатывать входные последовательности.

### Пример 2 (статья из Википедии):
**Документ:**
"Квантовые вычисления - это область компьютерных наук, использующая квантовые явления, такие как суперпозиция и запутанность. В отличие от классических компьютеров, использующих биты (0 или 1), квантовые компьютеры используют кубиты, которые могут существовать в суперпозиции состояний."

**Вопрос (фактический):** Что используют квантовые компьютеры вместо битов?
**Ответ:** Квантовые компьютеры используют кубиты, которые могут существовать в суперпозиции состояний.

**Вопрос (сравнительный):** В чем отличие квантовых компьютеров от классических?
**Ответ:** В отличие от классических компьютеров, использующих биты (0 или 1), квантовые компьютеры используют кубиты, которые могут существовать в суперпозиции состояний.

## Примеры некорректной разметки

### Пример 1:
**Документ:** "Искусственные нейронные сети имитируют работу биологических нейронов."

**Вопрос (некорректный):** Как работают компьютеры?
**Ответ (некорректный):** Компьютеры работают с помощью процессоров.
**Проблема:** Вопрос и ответ не связаны с информацией в документе.

### Пример 2:
**Документ:** "Температура на Марсе колеблется от -153°C ночью до 20°C днем на экваторе."

**Вопрос (некорректный):** Какая температура на Марсе?
**Ответ (некорректный):** На Марсе холодно.
**Проблема:** Ответ слишком общий и не использует фактическую информацию из текста.

## Процесс выполнения задания
1. Откройте предоставленный CSV-файл с документами
2. Для каждой строки в столбце 'document_ru':
   - Создайте вопрос и запишите его в столбец 'question_ru'
   - Найдите ответ в тексте и запишите его в столбец 'answer_ru'
   - Определите тип вопроса и запишите его в столбец 'question_type'
3. Сохраните и отправьте размеченный файл обратно

## Оценка качества
Ваша работа будет проверена на соответствие требованиям. Особое внимание будет уделено:
1. Связи вопросов с содержанием текста
2. Точности и полноте ответов
3. Разнообразию типов вопросов
4. Отсутствию субъективных интерпретаций
"""

with open('qa_dataset_tz.md', 'w', encoding='utf-8') as f:
    f.write(tz_text)

# Результаты разметки

Обратная связь:

"""

Разметка примерно 40 документов заняла около 2-3 часов. Задача понятная, но требует внимательности. Инструкция в целом ясная, примеры полезные. Есть несколько вопросов:
- Вопросы "Сравните ... с другими понятиями" иногда тяжело формулировать по смыслу в принципе.
- Нужно больше примеров для "описательных" и особенно "сравнительных" вопросов

Сложности - во-первых, в некоторых научных текстах трудно найти материал для сравнительных вопросов. Во-вторых, не всегда очевидна граница между типами вопросов. Предложение - добавить критерии для определения типа вопроса


"""

После получения данных я их немного отформатировала, вернула колонку `correct_answer` и теперь их можно просмотреть в датасете:

In [8]:
labeled_data = pd.read_excel('hw3_markuped_dataset.xlsx', header=1)
labeled_data = labeled_data[(labeled_data["document_ru"] != None) & (labeled_data["answer_ru"] != None)]
print("Number of markuped data", len(labeled_data))
labeled_data.sample(5)

Number of markuped data 38


,document,title,document_ru,document_type,source,document_length,question,correct_answer,question_ru,answer_ru,is_test_case,question_length,question_type,answer,answer_length,answer_to_document_ratio
5,«Мир движется вперёд» (англ. The World Moves O...,Мир движется вперёд,«Мир движется вперёд» (англ. The World Moves O...,wikipedia,Wikipedia_new,1252.0,NaN,NaN,Сравните «Мир движется вперёд» с другими понят...,"Фильм ""Мир движется вперёд"" рассказывает о сем...",NaN,51.0,comparative,NaN,131.0,0.104633
23,Лиху́шино — деревня в Грязовецком районе Волог...,Лихушино,Лиху́шино — деревня в Грязовецком районе Волог...,wikipedia,Wikipedia_new,499.0,NaN,NaN,Сравните Лиху́шино — деревня с другими понятиями.,Лихушино - это маленькая деревня в российской ...,NaN,49.0,comparative,NaN,68.0,0.136273
31,Скотт Стю́арт Ба́кула (англ. Scott Stewart Bak...,"Бакула, Скотт",Скотт Стю́арт Ба́кула (англ. Scott Stewart Bak...,wikipedia,Wikipedia_new,1572.0,NaN,NaN,Опишите Скотт Стю́арт Ба́кула.,"Скотт — американский актер, старший сын в семь...",NaN,30.0,descriptive,NaN,74.0,0.047074
27,Памятник Петру Великому установлен в городе Ту...,Памятник Петру I (Тула),Памятник Петру Великому установлен в городе Ту...,wikipedia,Wikipedia_new,523.0,NaN,NaN,Опишите Памятник Петру Великому.,Памятник Петру Великому в Туле стоит перед зда...,NaN,32.0,descriptive,NaN,4.0,0.007648
36,Глубокое обучение — подраздел машинного обучен...,NaN,Глубокое обучение — подраздел машинного обучен...,test_case,validation,199.0,NaN,"Подраздел машинного обучения, основанный на ис...",Опишите Глубокое обучение —.,Оно позволяет моделям обучаться представлению ...,1.0,28.0,descriptive,NaN,87.0,0.437186


In [10]:
import random

print("Примеры разметки:")
sample_indices = []
attempts = 0
while len(sample_indices) < 5 and attempts < 20:
    idx = random.randint(0, len(labeled_data) - 1)
    row = labeled_data.iloc[idx]
    
    if isinstance(row.get('question_ru'), str) and isinstance(row.get('answer_ru'), str):
        if row['question_ru'] and row['answer_ru']:
            sample_indices.append(idx)
    
    attempts += 1

for idx in sample_indices:
    row = labeled_data.iloc[idx]
    
    doc = row.get('document_ru', '')
    doc_preview = str(doc)# [:100] + "..." if isinstance(doc, str) and len(str(doc)) > 100 else str(doc)

    print(f"\nДокумент [{idx}]: {doc_preview}")
    print(f"\nВопрос: {row.get('question_ru')}")
    print(f"\nОтвет: {row.get('answer_ru')}")
    print(f"\nТип вопроса: {row.get('question_type')}")
    print("-" * 100)

Примеры разметки:

Документ [19]: Антье Франк (нем. Antje Frank; род. 5 июня 1968[1][2], Зюдерхольц, Мекленбург — Передняя Померания) — немецкая гребчиха, выступавшая за сборные ГДР и объединённой Германии по академической гребле в конце 1980-х — начале 1990-х годов. Бронзовая призёрка летних Олимпийских игр в Барселоне, обладательница бронзовой медали чемпионата мира, победительница многих регат национального и международного значения.
 Антье Франк родилась 5 июня 1968 года в коммуне Зюдерхольц, ГДР. Проходила подготовку в Берлине в столичном гребном клубе «Динамо».
 Впервые заявила о себе в гребле в 1986 году, выиграв золотую медаль в распашных рулевых четвёрках на чемпионате мира среди юниоров в Чехословакии.

Вопрос: Опишите Антье Франк (нем..

Ответ: Впервые заявила о себе в гребле в 1986 году, выиграв золотую медаль в распашных рулевых четвёрках на чемпионате мира среди юниоров в Чехословакии.

Тип вопроса: descriptive
-------------------------------------------------------------

# Валидатор качества разметки

In [11]:
validation_results = validate_markup(labeled_data)
print("Результаты валидации:")
print(f"Ответы найдены в документе: {sum(validation_results['answer_in_document'])} из {len(validation_results['answer_in_document'])}")
print(f"Процент ответов в документе: {sum(validation_results['answer_in_document']) / max(1, len(validation_results['answer_in_document'])):.2%}")
print(f"Точность на тестовых случаях: {validation_results['test_case_accuracy']:.2%}")
print(f"Обнаружено выбросов по длине ответа: {len(validation_results['outliers'])}")
print(f"Обнаружено несоответствий: {len(validation_results['inconsistencies'])}")

Результаты валидации:
Ответы найдены в документе: 21 из 38
Процент ответов в документе: 55.26%
Точность на тестовых случаях: 66.67%
Обнаружено выбросов по длине ответа: 0
Обнаружено несоответствий: 18


# Вывод

1. Разметка QA датасета для научных текстов оказалась нетривиальной задачей по нескольким причинам:
- научные тексты часто содержат сложную терминологию, что затрудняет формулировку понятных вопросов
- создание разнообразных типов вопросов (фактических, описательных, сравнительных) требует глубокого понимания материала
- необходимость точного извлечения ответов из текста, избегая субъективных интерпретаций
- занимает много времени - в нашем эксперименте на разметку 40 документов потребовалось ~2 часа, что составляет ~3 минуты на один документ

2. Качество инструкции - разметчик отметил, что примеры были полезны, но потребовалось больше примеров для сравнительных вопросов.

3. Анализ качества разметки:

- в целом удалось разметить примерно половину (потому что сама разметка занимает много времени)
- точность на тестовых случаях может сначала показаться низкой, но с учетом упрощенного сравнения по конкретным *словам* (без учета синонимов и т.д.) и отсечки в то, что совпадать должно ровно 60% слов, то получилось достаточно неплохо.